## Importing libraries
We start of by importing the libraries that we need for this project.

In [13]:
from sklearn import preprocessing
import PIL
from PIL import Image
from tqdm import tqdm
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

## Case 1: images anomaly detection
When we look at the pictures we see alot of dishes and interior/exterior photo's but there are also alot of unhelpfull pictures. These pictures include logo's, people and other random images. There isn't any benifit of having these pictures on a tripadvisor page. Therefor we call these anomalies, these pictures have no need to be on a tripadvisor page, as they bring no information to the customer. Because of this it would be nice to detect these images, this model could then be used to clear up the pages of tripadvisor. 

The detection of these images is called anomaly detection, there are 2 sorts of anomaly detection:
- **Outlier detection:** For this model we need a dataset with both standard pictures and anomaly pictures. 
- **Novelty detection:** For this nodel the trainingset consists only of the standard pictures. The trainingset must be labeled, so it the models are in a supervised fashion.

The dataset of tripadvisor is not labeled, therefor we could use outlier detection. Another choice can be to create a dataset of interiors, exteriors and dishes. This dataset would allow us to use **novelty detection**. For this case we will start of by looking at novelty detection.



sources: 1

### Construction the dataset

The dataset will look like a array of 3 d matrixes.

In [35]:
# todo adding labels
def imagesInFolderToDataset(path, width, height, channels):
    fileNameList = glob.glob(f"{path}*")
    images = []
    for fileName in tqdm(fileNameList, total=len(fileNameList)):
        try:

            img = Image.open(f"{fileName}")
            img_np = np.array(img.resize(( width, height ), channels ))
            if img_np.shape == (width,height,channels):
                images.append(img_np)



        except PIL.UnidentifiedImageError:
            pass
    return np.array(images)




100%|██████████| 111/111 [00:00<00:00, 185.33it/s]


In [57]:
restaurants_train = imagesInFolderToDataset("Images/restaurant/", 64, 64, 3)
buffet_train = imagesInFolderToDataset("Images/buffet/", 64, 64, 3)

x_train = np.concatenate((restaurants_train, buffet_train), axis=0)

# normalize the data
x_train = x_train.astype(float) / 255.

100%|██████████| 111/111 [00:00<00:00, 158.61it/s]


Om te beginnen gaan we greyscale gebruiken, de reden hiervoor is dat we op deze manier de dimensionaliteit reduceren. Dit zorgt dat er minder kans is op overfitting. Fotos met rgb maken het wel mogelijk om een beter model te trainen maar hebben meer kans op overfitting, de oplossingen hiervoor zijn dim reduction of meer data.

In [60]:
x_train_greyscale = np.array(list(map( lambda x: np.dot(x[...,:3], [0.2989, 0.587, 0.114]), x_train)))



In [62]:
## convalutional autoencoder
class AnomalyDetector(Model):
  def __init__(self):
    super(AnomalyDetector, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Input(shape=(64, 64, 1)),
      layers.Conv2D(16, (3, 3), activation='relu', padding='same', strides=2),
      layers.Conv2D(8, (3, 3), activation='relu', padding='same', strides=2)])

    self.decoder = tf.keras.Sequential([
      layers.Conv2DTranspose(8, kernel_size=3, strides=2, activation='relu', padding='same'),
      layers.Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same'),
      layers.Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same')])


  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = AnomalyDetector()

autoencoder.compile(optimizer='adam', loss='mae')

In [64]:
history = autoencoder.fit(x_train_greyscale, x_train_greyscale, 
          epochs=20, 
          batch_size=128,
          shuffle=True)

Epoch 1/20
5/5 [==============================] - 1s 164ms/step - loss: 0.1609
Epoch 2/20
5/5 [==============================] - 1s 165ms/step - loss: 0.1444
Epoch 3/20
5/5 [==============================] - 1s 180ms/step - loss: 0.1308
Epoch 4/20
5/5 [==============================] - 1s 192ms/step - loss: 0.1201
Epoch 5/20
5/5 [==============================] - 1s 172ms/step - loss: 0.1102
Epoch 6/20
5/5 [==============================] - 1s 194ms/step - loss: 0.1017
Epoch 7/20
5/5 [==============================] - 1s 178ms/step - loss: 0.0952
Epoch 8/20
5/5 [==============================] - 1s 169ms/step - loss: 0.0904
Epoch 9/20
5/5 [==============================] - 1s 180ms/step - loss: 0.0872
Epoch 10/20
5/5 [==============================] - 1s 219ms/step - loss: 0.0851
Epoch 11/20
5/5 [==============================] - 1s 206ms/step - loss: 0.0835
Epoch 12/20
5/5 [==============================] - 1s 195ms/step - loss: 0.0821
Epoch 13/20
5/5 [==============================] 

Ze kunnen anomely detection opdelen in 2 verschillende delen:
- **Outlier detection:** Our input dataset contains examples of both standard events and anomaly events. These algorithms seek to fit regions of the training data where the standard events are most concentrated, disregarding, and therefore isolating, the anomaly events. Such algorithms are often trained in an unsupervised fashion (i.e., without labels). We sometimes use these methods to help clean and pre-process datasets before applying additional machine learning techniques.

- **Novelty detection:** Unlike outlier detection, which includes examples of both standard and anomaly events, novelty detection algorithms have only the standard event data points (i.e., no anomaly events) during training time. During training, we provide these algorithms with labeled examples of standard events (supervised learning). At testing/prediction time novelty detection algorithms must detect when an input data point is an outlier.

sources: 1



## Sources
1. https://pyimagesearch.com/2020/01/20/intro-to-anomaly-detection-with-opencv-computer-vision-and-scikit-learn/
2. https://www.tensorflow.org/tutorials/generative/autoencoder

https://www.guru99.com/autoencoder-deep-learning.html